In [1]:
import cobra
import pandas as pd
import os
from os.path import join
from cobra import Model, Reaction, Metabolite
from cobra.sampling import sampling
import numpy as np
os.environ["R_HOME"] = f"{os.environ['CONDA_PREFIX']}\\Lib\\R"
import rpy2.robjects
from plotnine import *
import matplotlib.pyplot as plt 

In [2]:
# Importing the model
model=cobra.io.read_sbml_model("C:\\Users\\Maive\\Desktop\\BSc_loputoo\\Model_files\\Rt_IFO0880.xml")
model.objective = "EX_glc__D_e" 

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-21


In [3]:
# Lab data
model.reactions.EX_glc__D_e.bounds = -9999, 9999
growth_rates = [0.049, 0.100, 0.151, 0.203, 0.25, 0.301]
solution = model.optimize('minimize')
# All fluxes
all_fluxes = solution.fluxes.to_frame(name='Flux')
all_fluxes.T['EX_glc__D_e']
solution.fluxes['EX_glc__D_e']

-9999.0

In [4]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,0.03155,6,100.00%
o2_e,EX_o2_e,0.1893,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-0.1893,1,100.00%
h2o_e,EX_h2o_e,-0.1893,0,0.00%


In [5]:
# Get all fluxes on different growth rates
all_fluxes_dif_GR = pd.DataFrame(columns=['Growth rate', *all_fluxes.index], index=range(len(growth_rates))) #flux_values.index gives the row names column, * extracts the list of strings

for i in range(len(growth_rates)):
    model.reactions.BIOMASS_RT.bounds = growth_rates[i], growth_rates[i]
    solution = model.optimize('maximize')
    all_fluxes_dif_GR.loc[i] = solution.fluxes[['BIOMASS_RT', *all_fluxes.index]].values

all_fluxes_dif_GR


,Growth rate,ALCD25yi,MTHFCm,AMPN,DAGCPTer_RT,PYRt2,NNDPRm,HMGCOASm,PDE4,PAPSR,...,EX_2hxmp_e,SALCNHe,EX_btn_e,BTNt2i,EX_fol_e,FOLt,NADtm,EX_pydxn_e,PYDXNtr,RIBFLVt2
0,0.049,0.0,0.0,0.0,0.0,0.0,0.0,-0.004731,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,-0.009656,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.151,0.0,0.0,0.0,0.0,0.0,0.0,-0.01458,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.203,0.0,0.0,0.0,0.0,0.0,0.0,-0.019601,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.25,0.0,0.0,0.0,0.0,0.0,0.0,-0.024139,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.301,0.0,0.0,0.0,0.0,0.0,0.0,-0.029064,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# Get specific metabolites with their fluxes
exchange_fluxes_table = all_fluxes_dif_GR[['EX_glc__D_e', 'EX_o2_e', 'EX_nh4_e','EX_so4_e','EX_pi_e', 'EX_co2_e']]
exchange_fluxes_table

,EX_glc__D_e,EX_o2_e,EX_nh4_e,EX_so4_e,EX_pi_e,EX_co2_e
0,-0.67801,-1.588429,-0.323414,-0.004912,-0.013045,1.772102
1,-1.351989,-3.051467,-0.660029,-0.010024,-0.026622,3.42631
2,-2.025992,-4.514644,-0.996644,-0.015136,-0.040198,5.080656
3,-2.71321,-6.006511,-1.33986,-0.020348,-0.054042,6.767441
4,-3.33435,-7.354929,-1.650074,-0.02506,-0.066554,8.292035
5,-4.008352,-8.818105,-1.986688,-0.030172,-0.080131,9.946381


In [7]:
PPP_fluxes = all_fluxes_dif_GR[['BIOMASS_RT', 'EX_glc__D_e', 'G6PDH2r', 'TKT1', 'TALA','TKT2', 'XPK', 'FPK','PYRDC','PDHm', 'FPK']]
# Change G6PDH2rp column name to oxpp 
PPP_fluxes = PPP_fluxes.rename(columns = {'G6PDH2r': 'oxpp'})
PPP_fluxes

,BIOMASS_RT,EX_glc__D_e,oxpp,TKT1,TALA,TKT2,XPK,FPK,PYRDC,PDHm,FPK
0,0.049,-0.67801,0.0,-0.110978,-0.110978,0.142985,0.0,0.263134,0.036463,0.593997,0.263134
1,0.1,-1.351989,0.0,-0.226485,-0.226485,0.291807,0.0,0.537009,0.096356,1.143949,0.537009
2,0.151,-2.025992,0.0,-0.341993,-0.341993,0.440628,0.0,0.810884,0.150668,1.691168,0.810884
3,0.203,-2.71321,0.0,-0.459765,-0.459765,0.592368,0.0,1.090128,0.206045,2.249117,1.090128
4,0.25,-3.33435,0.0,-0.566213,-0.566213,0.729517,0.0,1.342523,0.256097,2.753417,1.342523
5,0.301,-4.008352,0.0,-0.681721,-0.681721,0.878338,0.0,1.616397,0.310409,3.300636,1.616397


In [14]:
ATPM_ACITL_fluxes = all_fluxes_dif_GR[['BIOMASS_RT', 'EX_glc__D_e', 'ATPM', 'ACITL', 'ALCD2y']] #ACS - Acetyl-CoA synthetase
ATPM_ACITL_fluxes

,BIOMASS_RT,EX_glc__D_e,ATPM,ACITL,ALCD2y
0,0.049,-0.67801,1.22,0.0,0.0
1,0.1,-1.351989,1.22,0.0,0.0
2,0.151,-2.025992,1.22,0.0,0.0
3,0.203,-2.71321,1.22,0.0,0.0
4,0.25,-3.33435,1.22,0.0,0.0
5,0.301,-4.008352,1.22,0.0,0.0


In [9]:
# Get all exchange fluxes
# Make a pd dataframe with all exchange fluxes that are not zero, then make a pivot table with wanted metabolites fluxes on different growth rates

model.reactions.BIOMASS_RT.bounds = growth_rates[0], growth_rates[0]
solution = model.optimize('minimize')
exchange_fluxes_all = model.summary().to_frame()
exchange_fluxes_all['GR'] = growth_rates[0]


for i in range(1, len(growth_rates)):
    model.reactions.BIOMASS_RT.bounds = growth_rates[i], growth_rates[i]
    solution = model.optimize('minimize')
    model_summary = model.summary().to_frame()
    model_summary['GR'] = growth_rates[i]
    exchange_fluxes_all = pd.concat([exchange_fluxes_all, model_summary], axis=0)  
    
# exchange_fluxes_all = exchange_fluxes_all[(exchange_fluxes_all['flux']) != 0.0] # for getting non-zero fluxes only
exchange_fluxes_all['flux'] = abs(exchange_fluxes_all['flux'])

exchange_fluxes_all

,reaction,metabolite,factor,flux,GR
DM_3aap_c,DM_3aap_c,3aap_c,-1.0,0.034705,0.049
DM_4oglu_c,DM_4oglu_c,4oglu_c,-1.0,0.000068,0.049
DM_aacald_c,DM_aacald_c,aacald_c,-1.0,0.000000,0.049
DM_amob_m,DM_amob_m,amob_m,-1.0,0.000118,0.049
DM_bcaro_d,DM_bcaro_d,bcaro_d,-1.0,0.000000,0.049
...,...,...,...,...,...
SRC_trnaser_c,SRC_trnaser_c,trnaser_c,1.0,0.101398,0.301
SRC_trnathr_c,SRC_trnathr_c,trnathr_c,1.0,0.065657,0.301
SRC_trnatrp_c,SRC_trnatrp_c,trnatrp_c,1.0,0.007019,0.301
SRC_trnatyr_c,SRC_trnatyr_c,trnatyr_c,1.0,0.018771,0.301


In [10]:
# Get all non-zero exchange fluxes to a pivot table
exchange_fluxes_table_all = pd.pivot_table(exchange_fluxes_all, values='flux', index=['GR'], columns=['reaction'])

exchange_fluxes_table_all['EX_glc__D_e']

GR
0.049    0.678010
0.100    1.351989
0.151    2.025992
0.203    2.713210
0.250    3.334350
0.301    4.008352
Name: EX_glc__D_e, dtype: float64

In [11]:
# # Get all fluxes to excel
# with pd.ExcelWriter('C:\\Users\\Maive\\Desktop\\BSc_loputoo\\Results\\Simulated_fluxes\\Glucose_maximization\\all_fluxes_dif_GR.xlsx') as excel_writer:
#     all_fluxes_dif_GR.to_excel(excel_writer, sheet_name='Glucose uptake range 0.49-3.1', index=True)
    
# with pd.ExcelWriter('C:\\Users\\Maive\\Desktop\\BSc_loputoo\\Results\\Simulated_fluxes\\Glucose_maximization\\exchange_fluxes_dif_GR.xlsx') as excel_writer:
#     exchange_fluxes_table_all.to_excel(excel_writer, sheet_name='Exchange fluxes', index=True)

In [15]:
# #  Get all flux values separately for dif growth rates, make them to a csv file
# for i in range(len(growth_rates)):
#     all_fluxes_dif_GR.loc[i].to_csv(f'C:\\Users\\Maive\\Desktop\\BSc_loputoo\\Results\\Simulated_fluxes\\Glucose_maximization\\flux_values_specific_GR_{growth_rates[i]}.csv', index=True)